# Computing Allowed Hands for Quantum President
### Requirements
You must install Sage, instructions for which can be found [here](https://doc.sagemath.org/html/en/installation/conda.html). Alternatively you could upload this notebook to [CoCalc](https://cocalc.com/) which is certainly the best bet if you are on Windows as Sage otherwise then requires Cygwin.
### Define a ring of polynomials in $x$, $a$, $b$, $c$, $d$
$x$ is a placeholder variable, but $a$, $b$, $c$ and $d$ are the number of single cards, double cards, triple cards and quadruple cards (e.g. a quadruple could be if a hand contains all 4 aces) in the hand.

In [1]:
R.<x, a, b, c, d> = QQ[]

$f$ is the generating function for standard 4 player president. For each of the 13 card values we can draw between 0 and 4 cards, but notice (I missed it initially) the binomial coefficients, since we want to count the number of ways of getting e.g. 2 cards of a face value.

In [2]:
f = (1+4*a*x+6*b*x^2+4*c*x^3+d*x^4)^13

$g$ is the same but for 3 player president, where for one of the face values (as a card is excluded) we allow a max of 3 draws.

In [3]:
g = (1+4*a*x+6*b*x^2+4*c*x^3+d*x^4)^12 * (1+4*a*x+6*b*x^2+4*c*x^3)

$h$ is for 5 player president. Two cards are left out this time so we have to account for the cases where they are of different face value (first product) or the same face value (second product)

In [4]:
h = (1+4*a*x+6*b*x^2+4*c*x^3+d*x^4)^8 * (1+4*a*x+6*b*x^2+4*c*x^3)^2 + (1+4*a*x+6*b*x^2+4*c*x^3+d*x^4)^9 * (1+4*a*x+6*b*x^2)

The first function is adapted from [here](https://ask.sagemath.org/question/53323/extracting-terms-of-polynomial-of-certain-powers/) and takes the nth root of any monomial which is a perfect nth power. The second function is a hack to print a given polynomial in descending order of the coefficents of its terms. compute_for_generator takes a given generating function and finds the coefficient (in a,b,c,d) of $x^n$ where $n$ is the hand size (called "take"). If div is set to true it then divides by the total if all the free variables are set to 1 (which counts how many hands of size $n$ there are). We then raise the coefficient (a polynomial in $a$, $b$, $c$ and $d$) to the power of the number of players and extract the perfect power monomials and root them. If div is true we also multiply by $100.0$ to get percentage probabilities. Finally, order the terms as before.

In [9]:
def only_perfect_power_monomials(f, n):
    R = f.parent()
    def nth_root_or_zero(m):
        try:
            return m^(1/n)
        except ValueError:
            return R.zero()
    mm = f.monomials()
    return sum([f[m] * nth_root_or_zero(m) for m in mm], R.zero())

def order_by_terms(p):
    def get_coeff(mon):
        try:
            return float(mon.split("*")[0])
        except ValueError:
            return 1
    return " + ".join(sorted(str(p).split(" + "), reverse=True, key=get_coeff))

def compute_for_generator(f, take=13, players=4, div=True):
    take_per_player = f.coefficient({x:take})
    if div:
        take_per_player_prob = take_per_player / take_per_player.substitute({a:1,b:1,c:1,d:1})
    else:
        take_per_player_prob = take_per_player
    take_overall = take_per_player_prob^players
    if div:
        probs = 100.0 * only_perfect_power_monomials(take_overall, players)
    else:
        probs = only_perfect_power_monomials(take_overall, players)
    return order_by_terms(probs)

For 4 players, the most likely outcome is 6 singles, 2 doubles and a triple, with 1.723% chance.`

In [10]:
compute_for_generator(f)

'1.72252240762150*a^6*b^2*c + 0.877581202511554*a^7*b^3 + 0.746835361243830*a^5*b^4 + 0.563968725643764*a^4*b^3*c + 0.0205570113706279*a^8*b*c + 0.00927014255365473*a^9*b^2 + 0.00399018350624033*a^3*b^5 + 0.00176910665219154*a^5*b*c^2 + 0.000415596210818873*a^2*b^4*c + 0.000342931110435850*a^3*b^2*c^2 + 0.0000274869246698372*a^5*b^2*d + 2.68505724947749e-6*a^7*b*d + 2.51865544510790e-6*a^7*c^2 + 7.74823291384573e-7*a^3*b^3*d + 4.98291387225133e-7*a^11*b + 4.93223959579481e-7*a^4*b*c*d + 1.47299837369773e-7*a^10*c + 1.97491650572249e-8*a*b^3*c^2 + 1.55309947422104e-8*a*b^6 + 6.24154480889012e-9*a^6*c*d + 5.37861060449645e-9*a^2*b^2*c*d + 9.02777682483352e-10*a^4*c^3 + 6.69166261360306e-10*a^2*b*c^3 + 2.04603958530723e-10*a^9*d + 1.03568161807245e-11*a*b^4*d + 6.46855257901308e-12*b^5*c + 4.10642295854319e-13*a^3*c^2*d + 1.24734776469882e-14*a^13 + 6.47984212014216e-15*a*b*c^2*d + 6.44753749823362e-15*a^3*b*d^2 + 1.22675024782278e-15*a^5*d^2 + 4.75457924995811e-16*b^2*c^3 + 4.07185138578

For 3 players, the most likely outcome is 6 singles, 4 doubles and a triple.

In [6]:
compute_for_generator(g, take=17, players=3)

'4.18073597543673*a^6*b^4*c + 1.75741108227523*a^4*b^5*c + 1.43056343789123*a^5*b^3*c^2 + 0.207533649763542*a^7*b^2*c^2 + 0.165545877828272*a^8*b^3*c + 0.136422350985267*a^5*b^6 + 0.135582575958755*a^3*b^4*c^2 + 0.0549635687132487*a^7*b^5 + 0.0104893316922015*a^4*b^2*c^3 + 0.00808469918256861*a^2*b^6*c + 0.00796750504396531*a^6*b^2*c*d + 0.00760642565423642*a^4*b^3*c*d + 0.00499782269309869*a^3*b^7 + 0.00452572228178032*a^5*b^4*d + 0.00266414771043243*a^6*b*c^3 + 0.00167950834666751*a^7*b^3*d + 0.000322835211526758*a^9*b*c^2 + 0.000240012869488548*a^2*b^3*c^3 + 0.000212163813080206*a^3*b^5*d + 0.000139256595701970*a^9*b^4 + 0.000111859908653751*a^5*b*c^2*d + 0.000106395610158253*a^8*b*c*d + 0.0000673444727292290*a^2*b^4*c*d + 0.0000594983167733847*a^3*b^2*c^2*d + 0.0000513612376113944*a*b^5*c^2 + 0.0000307283510300481*a^10*b^2*c + 0.0000112463693577719*a^9*b^2*d + 1.91765920851298e-6*a^8*c^3 + 1.74570643973651e-6*a^3*b*c^4 + 7.85088317982917e-7*a*b^8 + 4.89128276373072e-7*a^7*c^2*d + 2

For 5 players, the most likely outcome is 4 singles and 3 doubles.

In [7]:
compute_for_generator(h, take=10, players=5)

'1.33378172596462*a^4*b^3 + 0.472755424121613*a^6*b^2 + 0.140131051995963*a^3*b^2*c + 0.135991512062344*a^5*b*c + 0.00108648099316496*a^2*b^4 + 0.0000717072544285828*a^8*b + 3.45458088949471e-6*a*b^3*c + 2.18375325892043e-6*a^7*c + 4.09488559467802e-7*a^2*b*c^2 + 8.87334143941016e-8*a^4*c^2 + 8.19252894302661e-8*a^4*b*d + 1.59411627785219e-9*a^2*b^2*d + 5.68940899939683e-11*a^6*d + 7.42194281215510e-12*b^5 + 1.78858023721327e-12*a^3*c*d + 3.25636596246562e-13*a^10 + 4.43375261521789e-14*a*b*c*d + 3.05429745356177e-14*b^2*c^2 + 8.99039544472946e-17*b^3*d + 5.24112509586277e-17*a*c^3 + 1.55551284510579e-22*a^2*d^2 + 3.86502559505920e-25*c^2*d + 2.19629017439201e-27*b*d^2'

## Suit Division
This code works out the probability of the distributions of suits awarded. E.g. the most likely option is 4 cards each for 2 suits followed by 3 cards of another suit and 2 cards of the 4th suit with prob 21.551% which agrees with the more hand-cranked results from [here](http://www.rpbridge.net/7z77.htm).

In [11]:
R.<y, k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12, k13> = QQ[]

In [12]:
arr = [1, k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12, k13]
f = sum([binomial(13, i)*arr[i]*y^i for i in range(len(arr))])^4
order_by_terms(100.0 * f.coefficient({y:13})/f.substitute({k1:1, k2:1, k3:1, k4:1, k5:1, k6:1, k7:1, k8:1, k9:1, k10:1, k11:1, k12:1, k13:1}).coefficient({y:13}))

'21.5511756451633*k2*k3*k4^2 + 15.5168464645176*k2*k3^2*k5 + 12.9307053870980*k1*k3*k4*k5 + 10.5796680439893*k2^2*k4*k5 + 10.5361303154132*k3^3*k4 + 5.64248962346095*k2^2*k3*k6 + 4.70207468621746*k1*k2*k4*k6 + 3.44818810322613*k1*k3^2*k6 + 3.17390041319678*k1*k2*k5^2 + 2.99321883960602*k1*k4^3 + 1.88082987448698*k1*k2*k3*k7 + 1.32622619354851*k3*k4*k6 + 1.24333705645173*k4^2*k5 + 0.895202680645247*k3*k5^2 + 0.705311202932619*k1^2*k5*k6 + 0.651056495014725*k2*k5*k6 + 0.512953602132813*k2^3*k7 + 0.391839557184788*k1^2*k4*k7 + 0.361698052785958*k2*k4*k7 + 0.265245238709703*k3^2*k7 + 0.192357600799805*k1*k2^2*k8 + 0.117551867155436*k1^2*k3*k8 + 0.108509415835787*k1*k5*k7 + 0.108509415835787*k2*k3*k8 + 0.0723396105571916*k1*k6^2 + 0.0452122565982448*k1*k4*k8 + 0.0178108889629449*k1^2*k2*k9 + 0.0100471681329433*k1*k3*k9 + 0.00822041029058996*k2^2*k9 + 0.00556458542747628*k6*k7 + 0.00313007930295541*k5*k8 + 0.00109605470541199*k1*k2*k10 + 0.000966073858936854*k4*k9 + 0.000395797532509887*k1^3